In [5]:
import os
# print(os.getcwd())

import numpy as np
import matplotlib.pyplot as plt

from src import mlp
from src import mlplocker
from src import mnist
from src import config

In [6]:
# Importing trained MLP model.
W1 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'w1.npy'))
W2 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'w2.npy'))
W3 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'w3.npy'))
W4 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'w_out.npy'))

b1 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'b1.npy'))
b2 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'b2.npy'))
b3 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'b3.npy'))
b4 = np.load(os.path.join(config.WEIGHTS_AND_BIASES_PATH + r'\mlp_0\npy', 'b_out.npy'))

# Logic locking

In [7]:
trained_mlp = mlp.MLP(  num_classes = config.NUM_CLASSES, learning_rate = config.LEARNING_RATE,
                    W1 = W1, W2 = W2, W3 = W3, W4 = W4,
                    b1 = b1, b2 = b2, b3 = b3, b4 = b4)
mnist = mnist.MNIST(config.TRAINING_IMAGES_FILEPATH,
              config.TRAINING_LABELS_FILEPATH,
              config.TEST_IMAGES_FILEPATH,
              config.TEST_LABELS_FILEPATH)
locking = mlplocker.MLPLocker(trained_mlp)

MNIST Data loaded and processed:
  x_train shape: (60000, 784), dtype: float32
  y_train shape: (60000, 10), dtype: float64
  x_test shape: (10000, 784), dtype: float32
  y_test shape: (10000, 10), dtype: float64


In [ ]:
locked_accuracies = {}
unlocked_accuracies = {}
tests = np.linspace(0, config.N_TESTS_PER_LL_APPROACH - 1, config.N_TESTS_PER_LL_APPROACH)

locked_accuracy = np.zeros(config.N_TESTS_PER_LL_APPROACH)
unlocked_accuracy = np.zeros(config.N_TESTS_PER_LL_APPROACH)

def convert_binary_string_to_numpy_array(binary_string):
    """
    Converts a 128-bit binary string into a NumPy array of 8 16-bit integer values.
    """
    chunk_size = 16
    num_chunks = 8
    expected_length = chunk_size * num_chunks

    if len(binary_string) != expected_length:
        print(f"Error: Input binary string must be {expected_length} bits long.")
        return None

    integer_parts = []

    for i in range(num_chunks):
        start_index = i * chunk_size
        end_index = start_index + chunk_size
        binary_chunk = binary_string[start_index:end_index]
        decimal_value = int(binary_chunk, 2)
        integer_parts.append(decimal_value)

    return np.array(integer_parts, dtype=np.uint16)

fixed_binary_key = '00001101111001110010000100001110001010010111100000010011000100111100001001111010101001000000000111100100110010011100000001011110'
LOCKING_KEY = convert_binary_string_to_numpy_array(fixed_binary_key)

print(LOCKING_KEY)

locking.lock(LOCKING_KEY, 'permutation_2')
locked_accuracy = locking.test_locking(mnist.x_test, mnist.y_test)

locking.unlock(LOCKING_KEY)
unlocked_accuracy = locking.test_locking(mnist.x_test, mnist.y_test)
    
print(f'Locked accuracy: {locked_accuracy}')
print(f'Unlocked accuaracy: {unlocked_accuracy}')

[ 3559  8462 10616  4883 49786 41985 58569 49246]
Locked accuracy: 0.1135
Unlocked accuaracy: 0.9666
